Importing all the necessary libraries and creating an instance of the spotipy using our client secret and client id

In [96]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
import os

client_id = "72c9f2b9d3344c91af50a24f4144228f"
client_secret = "249943a9bbb74b9a8bf04bb7fd93790d"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 


Reading our data using pandas and displaying some random entries


In [97]:
data = pd.read_excel('./new_data.xlsx')

The data in the excel sheet is indexed from 1211 to 57167 hende taking only those indices as our primary work data

In [98]:
data = data[1211:57167]

Creating a list of all the artists in our dataset (unique values using a set)

In [99]:
unique_artists = set(data['Artist'])
all_artists = list(unique_artists)
print(f' We have {len(all_artists)} unique artists in our dataset')

 We have 6104 unique artists in our dataset


Some of these artist names have no data hence they will throw an index error when we try to extract its URI using our API.
For example the index 14 from the all_artists list has no value. The Spotify response for artists which have data is shown below it

In [102]:
x = sp.search(all_artists[14])
x

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=T%E2%80%9A%C3%84%C3%A3he+White+Noise&type=track&offset=0&limit=10',
  'items': [],
  'limit': 10,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [103]:
y = sp.search(all_artists[0]) # Data for this artist is available hence we get the desired response unlike the above cell.
y

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Big+Daddy+Kane&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6futYSDVulYR2PktBjTB5W'},
       'href': 'https://api.spotify.com/v1/artists/6futYSDVulYR2PktBjTB5W',
       'id': '6futYSDVulYR2PktBjTB5W',
       'name': 'Big Daddy Kane',
       'type': 'artist',
       'uri': 'spotify:artist:6futYSDVulYR2PktBjTB5W'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE'

In [104]:
len(all_artists)

6104

The condition we need to check is that x['tracks']['items'] should be not empty. Hence we will append None for such artists whose data is not available

In [80]:
#Sample data with 300 artists
artist_300 = all_artists[:300]

Extracting artist uris of all the artists from our data (testing with 30) and creating a list of the same

In [81]:
import time
import numpy as np
artist_uris = []
sleep_min = 2
sleep_max = 5
request_count = 0
start_time = time.time()
for name in range(len(artist_300)):
    result = sp.search(artist_300[name])
    if result['tracks']['items']:
        artist_uris.append(result['tracks']['items'][0]['artists'][0]['uri']) # appending the extracted artist uris to our list 'artist_uris'
    else:
        artist_uris.append(None) # appending None for all the artists whose data is not available
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))




5 artists uris fetched
Loop #: 5
Elapsed Time: 4.404490947723389 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 9.581078052520752 seconds
15 artists uris fetched
Loop #: 15
Elapsed Time: 14.175628900527954 seconds
20 artists uris fetched
Loop #: 20
Elapsed Time: 19.22171902656555 seconds
25 artists uris fetched
Loop #: 25
Elapsed Time: 22.84555220603943 seconds
30 artists uris fetched
Loop #: 30
Elapsed Time: 26.143970012664795 seconds
35 artists uris fetched
Loop #: 35
Elapsed Time: 29.629687070846558 seconds
40 artists uris fetched
Loop #: 40
Elapsed Time: 33.46802616119385 seconds
45 artists uris fetched
Loop #: 45
Elapsed Time: 37.579187870025635 seconds
50 artists uris fetched
Loop #: 50
Elapsed Time: 42.147841930389404 seconds
55 artists uris fetched
Loop #: 55
Elapsed Time: 45.7289981842041 seconds
60 artists uris fetched
Loop #: 60
Elapsed Time: 49.16603493690491 seconds
65 artists uris fetched
Loop #: 65
Elapsed Time: 52.04860210418701 seconds
70 artists uris fetched

In [82]:
dataframe = pd.DataFrame.from_dict(artist_uris)
dataframe.to_csv("300_artists_uris.csv")

Taking 10 artists from the sample 300 data

In [94]:
artist_uris_10 = artist_uris[:10]
artists_10 =  artist_300[:10]

Creating a folder for every different artist 

Last 10 here

In [116]:
import os
os.mkdir(f'downloaded_songs')
for items in artist_300[:10]:
    os.mkdir(f'downloaded_songs/{items}')

Getting the song uris of top 10 songs for the above 10 artists

In [235]:
req_count = 0
start_time = time.time()
top_10_track_uris = []
top_10_track_names = []
top_10_track_ids = []
for i,uri in enumerate(artist_uris_10):
    results = sp.artist_top_tracks(uri)
    req_count += 1
    for index,track in enumerate(results['tracks'][:10]):
        if(track['preview_url'] is not None): 
            # print('track    : ' + track['name'])
            # print('audio    : ' + track['preview_url'])
            top_10_track_names.append(track['name'])
            top_10_track_uris.append([artists_10[i],track['preview_url']])
            top_10_track_ids.append(track['id'])
        else:
            continue

        if req_count % 5 == 0:
            print(str(req_count) + " artists uris fetched")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            print('Loop #: {}'.format(req_count))
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))
    


5 artists uris fetched
Loop #: 5
Elapsed Time: 4.525915145874023 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 7.618905067443848 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 11.14402723312378 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 14.400511026382446 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 18.88183307647705 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 22.26096796989441 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 25.397680044174194 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 27.97236394882202 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 30.533847093582153 seconds
5 artists uris fetched
Loop #: 5
Elapsed Time: 34.124920129776 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 39.213550090789795 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 44.04418420791626 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 46.59556984901428 seconds
10 artists uris fetched
Loop #: 10
Elapsed 

In [176]:
top_10_track_uris[:5] # Appended the name of artist with every song uris in order to save the files systematicaly in the future

[['Big Daddy Kane',
  'https://p.scdn.co/mp3-preview/263207856276f49988ad76b4dc0262cee5dff772?cid=72c9f2b9d3344c91af50a24f4144228f'],
 ['Big Daddy Kane',
  'https://p.scdn.co/mp3-preview/e1b164e8ca1c4c1235ec2a000ba902096a3ad7e5?cid=72c9f2b9d3344c91af50a24f4144228f'],
 ['Big Daddy Kane',
  'https://p.scdn.co/mp3-preview/25ffb206d1e9274f109ee83e9d0ef6fb3b78c294?cid=72c9f2b9d3344c91af50a24f4144228f'],
 ['Big Daddy Kane',
  'https://p.scdn.co/mp3-preview/e85f2e9599c3c6b766869b75e9dd1dc706575999?cid=72c9f2b9d3344c91af50a24f4144228f'],
 ['Big Daddy Kane',
  'https://p.scdn.co/mp3-preview/5314f53b5b0465126d3dc798ee91d7c113a5444e?cid=72c9f2b9d3344c91af50a24f4144228f']]

In [177]:
top_10_track_uris[:3],top_10_track_names[:3]

([['Big Daddy Kane',
   'https://p.scdn.co/mp3-preview/263207856276f49988ad76b4dc0262cee5dff772?cid=72c9f2b9d3344c91af50a24f4144228f'],
  ['Big Daddy Kane',
   'https://p.scdn.co/mp3-preview/e1b164e8ca1c4c1235ec2a000ba902096a3ad7e5?cid=72c9f2b9d3344c91af50a24f4144228f'],
  ['Big Daddy Kane',
   'https://p.scdn.co/mp3-preview/25ffb206d1e9274f109ee83e9d0ef6fb3b78c294?cid=72c9f2b9d3344c91af50a24f4144228f']],
 ["Ain't No Half-Steppin'", 'Smooth Operator', 'Warm It Up, Kane'])

File names with a '/' caused probelms while saving hence removing a '/' from our file names

In [178]:
top_10_track_names_clean = []
for name in top_10_track_names:
    if name is not None:
        name = name.replace('/','')
        top_10_track_names_clean.append(name)

In [117]:
start_time = time.time()
r = 0

for i in range(len(artists_10)):
    for song in range(len(top_10_track_uris)):
        if top_10_track_uris[song] is not None:
            response = requests.get(top_10_track_uris[song][1], verify=False) # sending request with our song uris
            r+=1
            if artists_10[i] == top_10_track_uris[song][0]: # saving songs in folders named with artists
                open(f"downloaded_songs/{artists_10[i]}/{top_10_track_names_clean[song]}.mp3", 'wb').write(response.content)
            else:
                continue
        else:
            continue
        if r % 5 == 0:
            print(str(req_count) + " songs downloaded")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            print('Loop #: {}'.format(req_count))
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))   

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

10 songs downloaded
Loop #: 10
Elapsed Time: 4.150847911834717 seconds


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

10 songs downloaded
Loop #: 10
Elapsed Time: 51.872331857681274 seconds


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

...

10 songs downloaded
Loop #: 10
Elapsed Time: 260.11011505126953 seconds


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

10 songs downloaded
Loop #: 10
Elapsed Time: 319.115700006485 seconds


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'p.scdn.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

10 songs downloaded
Loop #: 10
Elapsed Time: 323.7185869216919 seconds


DATA

In [283]:
df = pd.DataFrame(top_10_track_names_clean, columns=['Song Name'])
df1 = pd.DataFrame(top_10_track_uris, columns=['Artist','Song Uri'])
df2 = pd.DataFrame(top_10_track_ids, columns=['track_id'])

In [284]:
df_combined = pd.concat([df1, df,df2], axis=1, join='outer')

In [285]:
df_combined

,Artist,Song Uri,Song Name,track_id
0,Big Daddy Kane,https://p.scdn.co/mp3-preview/263207856276f499...,Ain't No Half-Steppin',0dNiLb9FEHrRK7VFDJctiR
1,Big Daddy Kane,https://p.scdn.co/mp3-preview/e1b164e8ca1c4c12...,Smooth Operator,1Z4mVNtYGnpjHUoZVo3IMU
2,Big Daddy Kane,https://p.scdn.co/mp3-preview/25ffb206d1e9274f...,"Warm It Up, Kane",6l1tmVO6bRmV7kSMsYaHZm
3,Big Daddy Kane,https://p.scdn.co/mp3-preview/e85f2e9599c3c6b7...,Terra N Ya Era,5VlvjTZGEosS3uq6eKqiiN
4,Big Daddy Kane,https://p.scdn.co/mp3-preview/5314f53b5b046512...,Set It Off,6L0reMMEHvJsEX7kBo8Dm5
...,...,...,...,...
76,Chris Clark,https://p.scdn.co/mp3-preview/41fb45aefec98a15...,Stay With Me!,4QoarD42gB4PY9T4JefPPB
77,Chris Clark,https://p.scdn.co/mp3-preview/baa84c9e9333a6b9...,Boss Life!,4rx0NBSgwpGqPQBFiGhUmf
78,Chris Clark,https://p.scdn.co/mp3-preview/0a775b761c88b64c...,Don’t Give up Queen,1Pc8rCBziV7kGhBln49u8b
79,Chris Clark,https://p.scdn.co/mp3-preview/b27810d749ae2b9f...,Fierce of tha Real!,06k5mqznc1HKvok6AnrKjr


Extracting Features using the previosuly extarcted track ids of the songs

In [237]:
len(top_10_track_ids) # track ids to extract the features

81

Creating a feature dictionary to store our features

In [286]:
features_dict = {}
features_dict['acousticness'] = []
features_dict['danceability'] = []
features_dict['energy'] = []
features_dict['instrumentalness'] = []
features_dict['liveness'] = []
features_dict['loudness'] = []
features_dict['speechiness'] = []
features_dict['tempo'] = []
features_dict['valence'] = []



In [243]:
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for ids in top_10_track_ids:
    features = sp.audio_features(ids) # extracting the features using spotifys api
    features_dict['acousticness'].append(features[0]['acousticness'])
    features_dict['danceability'].append(features[0]['danceability'])
    features_dict['energy'].append(features[0]['energy'])
    features_dict['instrumentalness'].append(features[0]['instrumentalness'])
    features_dict['liveness'].append(features[0]['liveness'])
    features_dict['loudness'].append(features[0]['loudness'])
    features_dict['speechiness'].append(features[0]['speechiness'])
    features_dict['tempo'].append(features[0]['tempo'])
    features_dict['valence'].append(features[0]['valence'])
    request_count+=1
    if request_count % 5 == 0:
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))



Loop #: 5
Elapsed Time: 3.177456855773926 seconds
Loop #: 10
Elapsed Time: 6.2647528648376465 seconds
Loop #: 15
Elapsed Time: 11.499444007873535 seconds
Loop #: 20
Elapsed Time: 16.340616941452026 seconds
Loop #: 25
Elapsed Time: 20.294076919555664 seconds
Loop #: 30
Elapsed Time: 25.50825786590576 seconds
Loop #: 35
Elapsed Time: 30.19963812828064 seconds
Loop #: 40
Elapsed Time: 34.09170484542847 seconds
Loop #: 45
Elapsed Time: 37.22006678581238 seconds
Loop #: 50
Elapsed Time: 42.37716293334961 seconds
Loop #: 55
Elapsed Time: 47.92991805076599 seconds
Loop #: 60
Elapsed Time: 51.25449085235596 seconds
Loop #: 65
Elapsed Time: 54.447760820388794 seconds
Loop #: 70
Elapsed Time: 59.10534977912903 seconds
Loop #: 75
Elapsed Time: 64.61406683921814 seconds
Loop #: 80
Elapsed Time: 68.2049810886383 seconds


In [266]:
temp = list(zip(features_dict['acousticness'],features_dict['danceability'],features_dict['energy'],features_dict['instrumentalness'],features_dict['liveness'],features_dict['loudness'],features_dict['speechiness'],features_dict['tempo'],features_dict['valence']))

In [278]:
data_features = pd.DataFrame(temp, columns=['acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence'])
data_features

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0.005690,0.836,0.728,0.000819,0.0649,-12.058,0.2340,100.842,0.599
1,0.093300,0.891,0.319,0.000104,0.0482,-16.099,0.3330,92.649,0.687
2,0.095300,0.878,0.311,0.000000,0.0808,-17.401,0.3240,117.759,0.486
3,0.203000,0.747,0.865,0.003730,0.0882,-5.226,0.2960,92.995,0.773
4,0.000359,0.800,0.611,0.000000,0.2270,-13.294,0.2120,118.764,0.777
...,...,...,...,...,...,...,...,...,...
76,0.735000,0.531,0.335,0.031600,0.1040,-9.816,0.0484,170.022,0.301
77,0.101000,0.647,0.675,0.003500,0.2340,-8.099,0.0416,77.008,0.423
78,0.384000,0.900,0.388,0.000000,0.1030,-11.829,0.3410,135.029,0.455
79,0.022600,0.729,0.556,0.000000,0.4720,-10.806,0.3530,169.775,0.708


In [287]:
final_df = pd.concat([df_combined,data_features], axis=1, join='outer')
final_df

,Artist,Song Uri,Song Name,track_id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,Big Daddy Kane,https://p.scdn.co/mp3-preview/263207856276f499...,Ain't No Half-Steppin',0dNiLb9FEHrRK7VFDJctiR,0.005690,0.836,0.728,0.000819,0.0649,-12.058,0.2340,100.842,0.599
1,Big Daddy Kane,https://p.scdn.co/mp3-preview/e1b164e8ca1c4c12...,Smooth Operator,1Z4mVNtYGnpjHUoZVo3IMU,0.093300,0.891,0.319,0.000104,0.0482,-16.099,0.3330,92.649,0.687
2,Big Daddy Kane,https://p.scdn.co/mp3-preview/25ffb206d1e9274f...,"Warm It Up, Kane",6l1tmVO6bRmV7kSMsYaHZm,0.095300,0.878,0.311,0.000000,0.0808,-17.401,0.3240,117.759,0.486
3,Big Daddy Kane,https://p.scdn.co/mp3-preview/e85f2e9599c3c6b7...,Terra N Ya Era,5VlvjTZGEosS3uq6eKqiiN,0.203000,0.747,0.865,0.003730,0.0882,-5.226,0.2960,92.995,0.773
4,Big Daddy Kane,https://p.scdn.co/mp3-preview/5314f53b5b046512...,Set It Off,6L0reMMEHvJsEX7kBo8Dm5,0.000359,0.800,0.611,0.000000,0.2270,-13.294,0.2120,118.764,0.777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Chris Clark,https://p.scdn.co/mp3-preview/41fb45aefec98a15...,Stay With Me!,4QoarD42gB4PY9T4JefPPB,0.735000,0.531,0.335,0.031600,0.1040,-9.816,0.0484,170.022,0.301
77,Chris Clark,https://p.scdn.co/mp3-preview/baa84c9e9333a6b9...,Boss Life!,4rx0NBSgwpGqPQBFiGhUmf,0.101000,0.647,0.675,0.003500,0.2340,-8.099,0.0416,77.008,0.423
78,Chris Clark,https://p.scdn.co/mp3-preview/0a775b761c88b64c...,Don’t Give up Queen,1Pc8rCBziV7kGhBln49u8b,0.384000,0.900,0.388,0.000000,0.1030,-11.829,0.3410,135.029,0.455
79,Chris Clark,https://p.scdn.co/mp3-preview/b27810d749ae2b9f...,Fierce of tha Real!,06k5mqznc1HKvok6AnrKjr,0.022600,0.729,0.556,0.000000,0.4720,-10.806,0.3530,169.775,0.708


Extracting features from the 30s song previews (MFCCs and Delta MFCCs)